In [8]:
import gluonbook as gb
from mxnet import gluon,init,nd
from mxnet.gluon import nn
import mxnet as mx

In [9]:
def nin_block(num_channels,kernel_size,strides,padding):
    blk=nn.Sequential()
    blk.add(nn.Conv2D(num_channels,kernel_size,strides,padding,activation='relu'),
           nn.Conv2D(num_channels,kernel_size=1,activation='relu'),
           nn.Conv2D(num_channels,kernel_size=1,activation='relu'))
    return blk

In [10]:
net=nn.Sequential()
net.add(nin_block(96,kernel_size=11,strides=4,padding=0),
       nn.MaxPool2D(pool_size=3,strides=2),
       nin_block(256,kernel_size=5,strides=1,padding=2),
       nn.MaxPool2D(pool_size=3,strides=2),
       nin_block(384,kernel_size=3,strides=1,padding=1),
       nn.MaxPool2D(pool_size=3,strides=2),
       nn.Dropout(0.5),
       nin_block(10,kernel_size=3,strides=1,padding=1),
       nn.GlobalAvgPool2D(),
       nn.Flatten())

In [11]:
X=nd.random.uniform(shape=(1,1,224,224))
net.initialize(force_reinit=True)
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:',X.shape)

sequential6 output shape: (1, 96, 54, 54)
pool4 output shape: (1, 96, 26, 26)
sequential7 output shape: (1, 256, 26, 26)
pool5 output shape: (1, 256, 12, 12)
sequential8 output shape: (1, 384, 12, 12)
pool6 output shape: (1, 384, 5, 5)
dropout1 output shape: (1, 384, 5, 5)
sequential9 output shape: (1, 10, 5, 5)
pool7 output shape: (1, 10, 1, 1)
flatten1 output shape: (1, 10)


In [12]:
lr,num_epochs,batch_size,ctx=0.1,5,128,mx.gpu(0)
net.initialize(force_reinit=True,ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=gb.load_data_fashion_mnist(batch_size,resize=224)
gb.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)


MXNetError: [16:34:49] src/storage/./pooled_storage_manager.h:108: cudaMalloc failed: out of memory

Stack trace returned 10 entries:
[bt] (0) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x30756a) [0x7f4db0d4756a]
[bt] (1) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x307b91) [0x7f4db0d47b91]
[bt] (2) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x29d67b3) [0x7f4db34167b3]
[bt] (3) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x29db065) [0x7f4db341b065]
[bt] (4) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x440e64) [0x7f4db0e80e64]
[bt] (5) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x24f085d) [0x7f4db2f3085d]
[bt] (6) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x24f0d33) [0x7f4db2f30d33]
[bt] (7) /home/tomoya/.local/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x2468d49) [0x7f4db2ea8d49]
[bt] (8) /home/tomoya/.local/lib/pyth